# अध्याय 6 प्रश्न और उत्तर

- [आई. परिचय](#आई. परिचय)
- [II. पर्यावरण विन्यास](#II. पर्यावरण विन्यास)
- [III. वेक्टर डेटाबेस लोड हो रहा है](#III. वेक्टर डेटाबेस लोड हो रहा है)
- [IV. खोज-आधारित प्रश्न और उत्तर श्रृंखला का निर्माण](#IV. खोज-आधारित प्रश्न और उत्तर श्रृंखला का निर्माण)
- [वी. खोज-आधारित प्रश्न और उत्तर श्रृंखला की गहन खोज](#वी. खोज-आधारित प्रश्न और उत्तर श्रृंखला की गहन खोज)
- [5.1 खोज-आधारित प्रश्न और उत्तर चाटेम्पलेट पर आधारित](#5.1-टेम्पलेट पर आधारित खोज-आधारित प्रश्न और उत्तर श्रृंखला)
- [5.2 MapReduce पर आधारित खोज-आधारित प्रश्न और उत्तर श्रृंखला](#5.2-MapReduce- पर आधारित खोज-आधारित प्रश्न और उत्तर श्रृंखला)
- [5.3 खोज-आधारित प्रश्न और उत्तर श्रृंखला रिफाइन पर आधारित](#5.3-खोज-आधारित प्रश्न और उत्तर श्रृंखला -रिफाइन- पर आधारित)
- [VI. प्रयोग: स्थिति रिकॉर्ड](#VI. प्रयोग: स्थिति रिकॉर्ड)

## 1 परिचय

पिछले अध्याय में, हमने चर्चा की है कि किसी दिए गए प्रश्न से संबंधित दस्तावेज़ों को कैसे पुनः प्राप्त किया जाए। अगला कदम इन दस्तावेज़ों को लेना, मूल प्रश्न लेना, उन्हें भाषा मॉडल में एक साथ भेजना और इसमें प्रश्न का उत्तर देने के लिए कहना है बेशक, हम इस प्रक्रिया और इस कार्य को पूरा करने के कई अलग-अलग तरीकों के बारे में विस्तार से जानेंगे।

अब जब हमने संपूर्ण भंडारण और पुनर्प्राप्ति प्रक्रिया पूरी कर ली है और हमारे पास प्रासंगिक कटे हुए दस्तावेज़ हैं, तो अब हमें उत्तर पाने के लिए उन्हें भाषा मॉडल में पारित करने की आवश्यकता है। इस प्रक्रिया का सामान्य प्रवाह इस प्रकार है: सबसे पहले प्रश्न पूछा जाता है, फिर हम प्रासंगिक दस्तावेज़ ढूंढते हैं, फिर इन कटे हुए दस्तावेज़ों को सिस्टम प्रॉम्प्ट के साथ भाषा मॉडल में पास करते हैं और उत्तर प्राप्त करते हैं।

डिफ़ॉल्ट रूप से, हम सभी दस्तावेज़ स्लाइस को एक ही संदर्भ विंडो, यानी समान भाषा मॉडल सीए में पास करते हैंहालाँकि, इस समस्या को हल करने के कुछ अलग-अलग तरीके हैं, और उन सभी के फायदे और नुकसान हैं। अधिकांश फायदे इस तथ्य से आते हैं कि कभी-कभी बहुत सारे दस्तावेज़ हो सकते हैं, लेकिन आप उन सभी को पास नहीं कर सकते समान संदर्भ विंडो। MapReduce, Refine, और MapRerank तीन विधियाँ हैं जिनका उपयोग लघु संदर्भ विंडो की इस समस्या को हल करने के लिए किया जाता है। हम उन्हें इस पाठ्यक्रम में संक्षेप में प्रस्तुत करेंगे।

## 2. पर्यावरण विन्यास

पर्यावरण को कॉन्फ़िगर करने की विधि पहले जैसी ही है, इसलिए मैं इसे यहां नहीं दोहराऊंगा।

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


2 सितंबर, 2023 के बाद, GPT-3.5 API अपडेट किया जाएगा, इसलिए यहां समय निर्णय की आवश्यकता है

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo-0301


## 3. वेक्टर डेटाबेस लोड हो रहा है

In [5]:
# पहले से मौजूद वेक्टर डेटाबेस को लोड करें
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/cs229_lectures/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [6]:
# आप देख सकते हैं कि इसमें वे 209 दस्तावेज़ शामिल हैं जिन्हें हमने पहले विभाजित किया था
print(vectordb._collection.count())

209


हम एक प्रश्न के लिए वेक्टर खोज का परीक्षण कर सकते हैं। निम्नलिखित कोड समानता के आधार पर वेक्टर डेटाबेस की खोज करेगा और आपको k दस्तावेज़ लौटाएगा।

In [34]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [35]:
question = "这节课的主要话题是什么"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

## 4. खोज-आधारित प्रश्न-उत्तर श्रृंखला का निर्माण करना

लैंगचेन के आधार पर, हम एक पुनर्प्राप्ति-आधारित प्रश्न उत्तर श्रृंखला का निर्माण कर सकते हैं जो प्रश्न उत्तर देने के लिए GPT3.5 का उपयोग करता है, जो एक पुनर्प्राप्ति चरण के माध्यम से प्रश्न उत्तर देने की एक विधि है, हम इसे एक भाषा मॉडल में पास करके एक पुनर्प्राप्तिकर्ता के रूप में बना सकते हैं वेक्टर डेटाबेस। फिर, हम इसे प्रश्न के रूप में कॉल कर सकते हैं और उत्तर प्राप्त कर सकते हैं।

In [9]:
# ChatGPT3.5 का उपयोग करें, तापमान को 0 पर सेट करें
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [10]:
# खोज-आधारित प्रश्न-उत्तर श्रृंखला आयात करें
from langchain.chains import RetrievalQA

In [11]:
# खोज-आधारित प्रश्न-उत्तर श्रृंखला घोषित करें
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
# आप इस पद्धति का उपयोग प्रश्नों और उत्तरों को खोजने के लिए कर सकते हैं
question = "What are major topics for this class?"
result = qa_chain({"query": question})

In [13]:
result["result"]

'The major topic for this class is machine learning. Additionally, there may be some discussion on statistics and algebra as a refresher, and later in the quarter, there may be some discussion on extensions for the material covered in the main lectures.'

In [32]:
# आप इस पद्धति का उपयोग प्रश्नों और उत्तरों को खोजने के लिए कर सकते हैं
question = "这节课的主要话题是什么"
result = qa_chain({"query": question})

In [33]:
print(result["result"])

从这些上下文来看，这节课的主要话题包括课程信息、在线资源和线性代数。


## 5. खोज-आधारित प्रश्न-उत्तर श्रृंखला की अपनी खोज को गहरा करें

उपरोक्त कोड के साथ, हम एक सरल पुनर्प्राप्ति-आधारित प्रश्न-उत्तर श्रृंखला को कार्यान्वित कर सकते हैं, आइए विवरण में जाएं और देखें कि लैंगचेन इस पुनर्प्राप्ति-आधारित प्रश्न-उत्तर श्रृंखला में क्या करता है।

### 5.1 टेम्प्लेट-आधारित पुनर्प्राप्ति प्रश्न-उत्तर श्रृंखला

हम पहले एक प्रॉम्प्ट टेम्पलेट को परिभाषित करते हैं। इसमें नीचे दिए गए संदर्भ स्निपेट का उपयोग करने के बारे में कुछ निर्देश होते हैं, और फिर संदर्भ चर के लिए एक प्लेसहोल्डर होता है।

In [14]:
from langchain.prompts import PromptTemplate

#बिल्ड प्रॉम्प्ट
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [36]:
#中文版
from langchain.prompts import PromptTemplate

#बिल्ड प्रॉम्प्ट
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [37]:
#श्रृंखला चलाएँ
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [38]:
question = "Is probability a class topic?"

In [39]:
result = qa_chain({"query": question})

In [40]:
result["result"]

'Yes, probability is a class topic and the instructor assumes familiarity with basic probability and statistics.'

In [47]:
#中文版
question = "机器学习是其中一节的话题吗"

In [48]:
result = qa_chain({"query": question})

In [49]:
result["result"]

'是的，机器学习是其中一节的话题。感谢您的提问！'

In [50]:
result["source_documents"][0]

Document(page_content="So in this class, we've tried to convey to you a broad set of principl es and tools that will \nbe useful for doing many, many things. And ev ery time I teach this class, I can actually \nvery confidently say that af ter December, no matter what yo u're going to do after this \nDecember when you've sort of completed this  class, you'll find the things you learn in \nthis class very useful, and these things will be useful pretty much no matter what you end \nup doing later in your life.  \nSo I have more logistics to go over later, but let's say a few more words about machine \nlearning. I feel that machine learning grew out of  early work in AI, early work in artificial \nintelligence. And over the last — I wanna say last 15 or last 20 years or so, it's been viewed as a sort of growing new capability for computers. And in particular, it turns out \nthat there are many programs or there are many applications that you can't program by \nhand.  \nFor example, if you

यह दृष्टिकोण बहुत अच्छा है क्योंकि इसमें भाषा मॉडल के लिए केवल एक ही कॉल शामिल है, इसकी सीमाएं भी हैं, अर्थात्, यदि बहुत सारे दस्तावेज़ हैं, तो उन सभी को संदर्भ विंडो में फिट करना संभव नहीं हो सकता है दस्तावेज़ों पर सवालों के जवाब देने के लिए एक और तकनीक, जिसका नाम है MapReduce तकनीक।

### 5.2 MapReduce पर आधारित पुनर्प्राप्ति-आधारित प्रश्न-उत्तर श्रृंखला

MapReduce तकनीक में, मूल उत्तर प्राप्त करने के लिए प्रत्येक व्यक्तिगत दस्तावेज़ को पहले व्यक्तिगत रूप से भाषा मॉडल में भेजा जाता है, फिर, इन उत्तरों को भाषा मॉडल के लिए अंतिम कॉल के माध्यम से अंतिम उत्तर में जोड़ा जाता है, हालांकि इसमें भाषा मॉडल के लिए अधिक कॉल शामिल होती हैं। इसमें किसी भी संख्या में दस्तावेज़ों को संसाधित करने का लाभ है।

In [51]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [52]:
question = "Is probability a class topic?"
result = qa_chain_mr({"query": question})

In [53]:
result["result"]

'It is not clear from the given portion of the document whether probability is a class topic or not. The text only mentions that familiarity with basic probability and statistics is assumed as a prerequisite for the class.'

In [55]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
#中文版
question = "概率论是其中一节的话题吗"
result = qa_chain_mr({"query": question})
result["result"]

'根据给出的文件部分，没有提到概率论。'

जब हम पिछले प्रश्न को इस श्रृंखला के माध्यम से चलाते हैं, तो हम इस दृष्टिकोण के साथ दो समस्याएं देख सकते हैं। पहला, यह बहुत धीमा है। दूसरे, दिए गए दस्तावेज़ के इस भाग के आधार पर इस प्रश्न का कोई स्पष्ट उत्तर नहीं है। ऐसा शायद इसलिए है क्योंकि यह प्रत्येक दस्तावेज़ का अलग-अलग उत्तर दे रहा है, इसलिए यदि जानकारी दो दस्तावेज़ों के बीच फैली हुई है, तो यह सभी जानकारी को एक ही संदर्भ में कैप्चर नहीं कर रही है।

In [ ]:
#आयात
#os.environ["LANGCHAIN_TRACING_V2"] = "सत्य"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langचेन.प्लस"
#os.environ["LANGCHAIN_API_KEY"] = "..." # अपनी एपीआई कुंजी के साथ बिंदुओं को बदलें

हम उपरोक्त पर्यावरण चर को आयात कर सकते हैं और फिर MapReduce दस्तावेज़ श्रृंखला के विवरण का पता लगा सकते हैं, उदाहरण के लिए, उपरोक्त प्रदर्शन में, हमने वास्तव में भाषा मॉडल में चार अलग-अलग कॉल शामिल किए हैं, प्रत्येक दस्तावेज़ को चलाने के बाद, उन्हें अंतिम श्रृंखला में एक साथ जोड़ दिया जाता है , स्टफ़्ड दस्तावेज़ श्रृंखला, जो इन सभी उत्तरों को अंतिम कॉल में विलीन कर देती है।

### 5.3 परिष्कृत-आधारित पुनर्प्राप्ति प्रश्न-उत्तर श्रृंखला

हम इसी तरह श्रृंखला प्रकार को रिफाइन पर सेट कर सकते हैं। यह एक नई श्रृंखला प्रकार है। एक रिफाइन दस्तावेज़ श्रृंखला एक मैपरेडस श्रृंखला के समान है, जहां एलएलएम को प्रत्येक दस्तावेज़ के लिए एक बार बुलाया जाता है, लेकिन सुधार यह है कि हम अंतिम संकेत भेजते हैं। एलएलएम हर बार एक अनुक्रम होता है जो पिछली प्रतिक्रिया को नए डेटा के साथ जोड़ता है और एक परिष्कृत प्रतिक्रिया का अनुरोध करता है, इसलिए यह आरएनएन के समान एक अवधारणा है, जहां हम विभिन्न दस्तावेजों में वितरित की जा रही जानकारी की समस्या को हल करने के लिए संदर्भ को बढ़ाते हैं।

In [56]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
question = "Is probability a class topic?"
result = qa_chain_mr({"query": question})
result["result"]

'Based on the additional context provided, probability is assumed to be a prerequisite and not a main topic of the class. The instructor assumes that students are familiar with basic probability and statistics, including random variables, expectation, variance, and basic linear algebra. The class will not be very programming-intensive, but some programming will be done in MATLAB or Octave. The instructor will provide a refresher course on the prerequisites in some of the discussion sections. The class also assumes familiarity with basic linear algebra, including matrices, vectors, matrix multiplication, and matrix inverse. Most undergraduate linear algebra courses, such as Math 51, 103, Math 113, or CS205 at Stanford, are more than enough. The instructor will also review the prerequisites in some of the discussion sections.'

In [57]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
question = "概率论是其中一节的话题吗"
result = qa_chain_mr({"query": question})
result["result"]

'Based on the additional context provided, the instructor mentions that they will cover statistics and algebra in the discussion sections as a refresher, and will also use the discussion sections to go over extensions of the material taught in the main lectures. However, there is no explicit mention of probability theory being covered in the course. Therefore, the original answer still stands.'

आप देखेंगे कि यह परिणाम MapReduce श्रृंखला के परिणाम से बेहतर है, ऐसा इसलिए है क्योंकि परिष्कृत श्रृंखला का उपयोग करने से आप टुकड़ों में जानकारी को जोड़ सकते हैं, जो वास्तव में MapReduce श्रृंखला की तुलना में अधिक जानकारी को प्रोत्साहित करता है।

## 6. प्रयोग: स्थिति रिकार्डिंग

आइए यहां एक प्रयोग करें.

हम डिफ़ॉल्ट सामग्री का उपयोग करके एक क्यूए श्रृंखला बनाएंगे। आइए प्रश्न पूछें, क्या संभाव्यता सिद्धांत पाठ्यक्रम का विषय है? यह उत्तर देगा, संभाव्यता सिद्धांत एक पूर्वापेक्षा होनी चाहिए, फिर हमें यह मिलेगा एक उत्तर। इस पाठ्यक्रम के लिए आवश्यक शर्तों में कंप्यूटर विज्ञान और बुनियादी कंप्यूटर कौशल और सिद्धांतों का बुनियादी ज्ञान शामिल है। यह संभाव्यता के बारे में पिछले प्रश्न से संबंधित नहीं है।

In [58]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [59]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a topic in this class. The speaker assumes that students have familiarity with basic probability and statistics, and mentions that most undergraduate statistics classes will be more than enough preparation for this class.'

In [60]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites are needed because in this class, the instructor assumes that all students have a basic knowledge of computer science and knowledge of basic computer skills and principles. This includes understanding of big-O notation and other fundamental concepts. Without this basic knowledge, it may be difficult to understand the material covered in the class.'

In [62]:
question = "概率论是这节课的一个内容吗"
result = qa_chain({"query": question})
result["result"]

'是的，作者在文中提到了这门课程需要学生具备基本的概率论和统计学知识。'

In [63]:
question = "为什么需要具备这些知识"
result = qa_chain({"query": question})
result["result"]

'在这段上下文中，作者提到这些知识是这门课程的先决条件，因为这门课程涉及到机器学习的基本概念和算法，需要学生具备计算机科学和基本计算机技能和原理的基本知识。如果学生没有这些基础知识，可能会很难理解和应用机器学习算法。因此，学生需要具备这些知识才能更好地学习和应用机器学习。'

मूल रूप से, हम जिस श्रृंखला का उपयोग कर रहे हैं उसमें राज्य की कोई अवधारणा नहीं है। यह पिछले प्रश्नों या पिछले उत्तरों को याद नहीं रखता है। इसे प्राप्त करने के लिए, हमें स्मृति का परिचय देने की आवश्यकता है, जिस पर हम अगले भाग में चर्चा करेंगे।